# Импорт библиотек и данных

In [52]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from statistics import variance, pvariance, median

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

from clickhouse_driver import Client

In [53]:
true_stores = [2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
              16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
              30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
              43,  46,  47,  48,  49,  51,  52,  53,  55,  56,  57,  58, 59,
              65,  67,  68,  69,  70,  71,  73,  74,  75,  77,  78,  79, 80,
              81,  82,  83,  86,  87,  88,  89,  91,  92,  93,  94, 109, 110, 
              111, 114, 115, 117, 118, 119, 122, 123, 124, 126, 127, 128, 129, 
              134, 135, 136, 137, 138, 140, 141, 142, 143, 147, 150, 153, 155, 
              156, 163, 165, 169, 181]

# Настройка соединения с КликХаус

In [54]:
client = Client(host='rc1a-p5kvb7b24s4r8i5k.mdb.yandexcloud.net', 
                port=9000, database='default', user='admin', password='qC8rD9fC1nQ1NO8p')

# Получение таблицы метрик для всех магазинов за 3 месяца 

## 2022-11-01 - 2023-01-31

In [55]:
def get_core(store):
    query  = client.execute("""
    WITH
    models AS 
    (
    SELECT item AS line_item_id,
           name_ru AS new_class
    FROM assortment_ods.rms_product_attribute
    WHERE attribute = 'model_modelID'
    ), 
    --- 
    core_for_second AS 
    (
    SELECT store_id,
           line_item_id,
           ROUND(stddevPop(toInt128(col)),2) AS pvariance_delta_sku
    FROM 
        (SELECT sale_date,
                store_id,
                receipt_surrogate_id,
                line_item_id ,
                (sale_date - neighbor(sale_date,1,toDate('2001-01-01'))) AS delta,
                CASE
                    WHEN delta > 365 THEN 'не число'
                    WHEN delta < 0 THEN 'не число'
                    ELSE toString(delta)
                END AS col
        FROM 
            (SELECT DISTINCT l.opened_date AS sale_date,
                             l.store_id,
                             l.receipt_surrogate_id,
                             l.line_item_id
            FROM assortment_ods.dds_receipt_lines_public l
            LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
            WHERE l.opened_date BETWEEN '2022-11-01' -- начало интервала
                                    AND '2023-01-31' -- конец интервала
              AND l.line_type IN ('Sales', 'pickedUp orders')
              AND length(l.line_item_id) <= 8
              AND l.store_id = {}
            ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date desc) a
            ORDER BY  line_item_id desc, store_id desc, sale_date desc) C
        WHERE col != 'не число'
        GROUP BY  store_id, line_item_id
    ), 
    --- 
    core_for_fourth AS 
    (
    SELECT l.line_item_id,
            l.opened_date,
            l.store_id,
            SUM(line_turnover) AS turnover
    FROM assortment_ods.dds_receipt_lines_public l
    LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
    WHERE l.opened_date BETWEEN '2022-11-01' -- начало интервала
                            AND '2023-01-31' -- конец интервала
      AND l.line_type IN ('Sales', 'pickedUp orders')
      AND length(l.line_item_id) <= 8
      AND l.store_id = {}
    GROUP BY  l.line_item_id, l.opened_date, l.store_id
    ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date DESC 
    ), 
    --- 
    second_metrics AS 
    (
    SELECT core_for_second.store_id,
           core_for_second.line_item_id,
           models.new_class,
           core_for_second.pvariance_delta_sku,
           quantile(0.05)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS low_border, 
           quantile(0.95)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS high_border,
           CASE
               WHEN core_for_second.pvariance_delta_sku >= high_border THEN 'upper'
               WHEN core_for_second.pvariance_delta_sku < low_border THEN 'lower'
               ELSE 'middle'
           END AS label,
           CASE
               WHEN label = 'upper' THEN 0
               ELSE 1
           END AS core
    FROM core_for_second
    LEFT JOIN models ON core_for_second.line_item_id = models.line_item_id
    ORDER BY  core_for_second.store_id asc, core_for_second.line_item_id asc, models.new_class DESC
    ), 
    --- 
    fourth_metrics AS 
    (
    SELECT store_id,
           line_item_id,
           new_class,
           sales_days,
           lowers,
           quantile(0.15)(sales_days) OVER (PARTITION by store_id, new_class) AS border_days_model,
           CASE
               WHEN sales_days <= border_days_model THEN 0
               WHEN lowers > 0 THEN 0
               ELSE 1
           END AS core 
    FROM
        (SELECT store_id,
                line_item_id,
                new_class,
                sales_days,
                SUM(is_lower) AS lowers 
         FROM
             (SELECT store_id,
                     line_item_id,
                     new_class,
                     opened_date,
                     day_TO_ratio_sku,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id, label_TO) AS opened_dates,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id) AS sales_days,
                     opened_dates / sales_days * 100 AS perc_sales,
                     CASE
                         WHEN label_TO = 'lower' AND perc_sales >= (100-15) THEN 1
                         ELSE 0
                     END AS is_lower 
              FROM
                  (SELECT store_id,
                          line_item_id,
                          new_class,
                          opened_date,
                          TO_day_sku/ TO_day AS day_TO_ratio_sku,
                          quantile(0.15)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS low_border,
                          quantile(0.85)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS high_border,
                          CASE
                              WHEN day_TO_ratio_sku >= high_border THEN 'upper'
                              WHEN day_TO_ratio_sku < low_border THEN 'lower'
                              ELSE 'middle'
                          END AS label_TO
                   FROM
                       (SELECT store_id,
                               line_item_id,
                               new_class,
                               opened_date,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date) AS TO_day,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, line_item_id) AS TO_day_sku,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, new_class) AS TO_day_class 
                        FROM
                            (SELECT core_for_fourth.store_id AS store_id,
                                    core_for_fourth.line_item_id AS line_item_id,
                                    models.new_class AS new_class,
                                    core_for_fourth.opened_date,
                                    SUM(turnover) AS turnover
                             FROM core_for_fourth
                             LEFT JOIN models ON core_for_fourth.line_item_id = models.line_item_id
                             GROUP BY  core_for_fourth.store_id, core_for_fourth.line_item_id, models.new_class, core_for_fourth.opened_date) a) 
                   b) 
              c)
         d
    GROUP BY  store_id, line_item_id, new_class, sales_days) f) 
    ---
    SELECT s.store_id,
           s.line_item_id,
           s.new_class,
           s.label,
           s.pvariance_delta_sku,
           s.high_border,
           f.sales_days,
           f.lowers,
           f.border_days_model,
           CASE
               WHEN s.core = 1 AND f.core = 1 THEN 2
               WHEN s.core = 1 AND f.core = 0 THEN 2
               WHEN s.pvariance_delta_sku <= s.high_border *2 AND f.core = 1 THEN 2
               ELSE s.core + f.core
           END AS c
    FROM second_metrics AS s 
    FULL OUTER JOIN fourth_metrics AS f ON s.store_id = f.store_id
                                       AND s.line_item_id = f.line_item_id
                                       AND s.new_class = f.new_class
    ;
    """.format(store,store))
    return pd.DataFrame(query, columns=['store_id',
                                        'line_item_id',
                                        'new_class',
                                        'label',
                                        'pvariance_delta_sku',
                                        'high_border',
                                        'sales_days',
                                        'lowers',
                                        'border_days_model',
                                        'core'])

In [56]:
#создаем пустую таблицу для записи 
metrics_1_22_11_01__23_01_31 = pd.DataFrame()

# запускаем цикл по всем магазинам из списка
for store in true_stores:
    metrics_1_22_11_01__23_01_31 = pd.concat([metrics_1_22_11_01__23_01_31, get_core(store).query('store_id != 0')], ignore_index=True)

KeyboardInterrupt: 

In [6]:
metrics_1_22_11_01__23_01_31.groupby(['line_item_id'], as_index=False).core.sum().query('core == 218')

,line_item_id,core
60,10029915,218
99,10051768,218
100,10051784,218
101,10051805,218
102,10052162,218
...,...,...
132837,923659,218
132842,929270,218
132852,941127,218
132907,955431,218


In [7]:
metrics_1_22_11_01__23_01_31.to_csv('metrics_1_22_11_01__23_01_31.csv')

In [8]:
del(metrics_1_22_11_01__23_01_31)

## 2022-11-15 - 2023-02-15

In [9]:
def get_core(store):
    query  = client.execute("""
    WITH
    models AS 
    (
    SELECT item AS line_item_id,
           name_ru AS new_class
    FROM assortment_ods.rms_product_attribute
    WHERE attribute = 'model_modelID'
    ), 
    --- 
    core_for_second AS 
    (
    SELECT store_id,
           line_item_id,
           ROUND(stddevPop(toInt128(col)),2) AS pvariance_delta_sku
    FROM 
        (SELECT sale_date,
                store_id,
                receipt_surrogate_id,
                line_item_id ,
                (sale_date - neighbor(sale_date,1,toDate('2001-01-01'))) AS delta,
                CASE
                    WHEN delta > 365 THEN 'не число'
                    WHEN delta < 0 THEN 'не число'
                    ELSE toString(delta)
                END AS col
        FROM 
            (SELECT DISTINCT l.opened_date AS sale_date,
                             l.store_id,
                             l.receipt_surrogate_id,
                             l.line_item_id
            FROM assortment_ods.dds_receipt_lines_public l
            LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
            WHERE l.opened_date BETWEEN '2022-11-15' -- начало интервала
                                    AND '2023-02-15' -- конец интервала
              AND l.line_type IN ('Sales', 'pickedUp orders')
              AND length(l.line_item_id) <= 8
              AND l.store_id = {}
            ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date desc) a
            ORDER BY  line_item_id desc, store_id desc, sale_date desc) C
        WHERE col != 'не число'
        GROUP BY  store_id, line_item_id
    ), 
    --- 
    core_for_fourth AS 
    (
    SELECT l.line_item_id,
            l.opened_date,
            l.store_id,
            SUM(line_turnover) AS turnover
    FROM assortment_ods.dds_receipt_lines_public l
    LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
    WHERE l.opened_date BETWEEN '2022-11-15' -- начало интервала
                            AND '2023-02-15' -- конец интервала
      AND l.line_type IN ('Sales', 'pickedUp orders')
      AND length(l.line_item_id) <= 8
      AND l.store_id = {}
    GROUP BY  l.line_item_id, l.opened_date, l.store_id
    ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date DESC 
    ), 
    --- 
    second_metrics AS 
    (
    SELECT core_for_second.store_id,
           core_for_second.line_item_id,
           models.new_class,
           core_for_second.pvariance_delta_sku,
           quantile(0.05)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS low_border, 
           quantile(0.95)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS high_border,
           CASE
               WHEN core_for_second.pvariance_delta_sku >= high_border THEN 'upper'
               WHEN core_for_second.pvariance_delta_sku < low_border THEN 'lower'
               ELSE 'middle'
           END AS label,
           CASE
               WHEN label = 'upper' THEN 0
               ELSE 1
           END AS core
    FROM core_for_second
    LEFT JOIN models ON core_for_second.line_item_id = models.line_item_id
    ORDER BY  core_for_second.store_id asc, core_for_second.line_item_id asc, models.new_class DESC
    ), 
    --- 
    fourth_metrics AS 
    (
    SELECT store_id,
           line_item_id,
           new_class,
           sales_days,
           lowers,
           quantile(0.15)(sales_days) OVER (PARTITION by store_id, new_class) AS border_days_model,
           CASE
               WHEN sales_days <= border_days_model THEN 0
               WHEN lowers > 0 THEN 0
               ELSE 1
           END AS core 
    FROM
        (SELECT store_id,
                line_item_id,
                new_class,
                sales_days,
                SUM(is_lower) AS lowers 
         FROM
             (SELECT store_id,
                     line_item_id,
                     new_class,
                     opened_date,
                     day_TO_ratio_sku,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id, label_TO) AS opened_dates,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id) AS sales_days,
                     opened_dates / sales_days * 100 AS perc_sales,
                     CASE
                         WHEN label_TO = 'lower' AND perc_sales >= (100-15) THEN 1
                         ELSE 0
                     END AS is_lower 
              FROM
                  (SELECT store_id,
                          line_item_id,
                          new_class,
                          opened_date,
                          TO_day_sku/ TO_day AS day_TO_ratio_sku,
                          quantile(0.15)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS low_border,
                          quantile(0.85)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS high_border,
                          CASE
                              WHEN day_TO_ratio_sku >= high_border THEN 'upper'
                              WHEN day_TO_ratio_sku < low_border THEN 'lower'
                              ELSE 'middle'
                          END AS label_TO
                   FROM
                       (SELECT store_id,
                               line_item_id,
                               new_class,
                               opened_date,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date) AS TO_day,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, line_item_id) AS TO_day_sku,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, new_class) AS TO_day_class 
                        FROM
                            (SELECT core_for_fourth.store_id AS store_id,
                                    core_for_fourth.line_item_id AS line_item_id,
                                    models.new_class AS new_class,
                                    core_for_fourth.opened_date,
                                    SUM(turnover) AS turnover
                             FROM core_for_fourth
                             LEFT JOIN models ON core_for_fourth.line_item_id = models.line_item_id
                             GROUP BY  core_for_fourth.store_id, core_for_fourth.line_item_id, models.new_class, core_for_fourth.opened_date) a) 
                   b) 
              c)
         d
    GROUP BY  store_id, line_item_id, new_class, sales_days) f) 
    ---
    SELECT s.store_id,
           s.line_item_id,
           s.new_class,
           s.label,
           s.pvariance_delta_sku,
           s.high_border,
           f.sales_days,
           f.lowers,
           f.border_days_model,
           CASE
               WHEN s.core = 1 AND f.core = 1 THEN 2
               WHEN s.core = 1 AND f.core = 0 THEN 2
               WHEN s.pvariance_delta_sku <= s.high_border *2 AND f.core = 1 THEN 2
               ELSE s.core + f.core
           END AS c
    FROM second_metrics AS s 
    FULL OUTER JOIN fourth_metrics AS f ON s.store_id = f.store_id
                                       AND s.line_item_id = f.line_item_id
                                       AND s.new_class = f.new_class
    ;
    """.format(store,store))
    return pd.DataFrame(query, columns=['store_id',
                                        'line_item_id',
                                        'new_class',
                                        'label',
                                        'pvariance_delta_sku',
                                        'high_border',
                                        'sales_days',
                                        'lowers',
                                        'border_days_model',
                                        'core'])

In [10]:
#создаем пустую таблицу для записи 
metrics_2_22_11_15__23_02_15 = pd.DataFrame()

# запускаем цикл по всем магазинам из списка
for store in true_stores:
    metrics_2_22_11_15__23_02_15 = pd.concat([metrics_2_22_11_15__23_02_15, get_core(store).query('store_id != 0')], ignore_index=True)

In [11]:
metrics_2_22_11_15__23_02_15.groupby(['line_item_id'], as_index=False).core.sum().query('core == 218')

,line_item_id,core
102,10051768,218
103,10051784,218
104,10051805,218
105,10052162,218
106,10052197,218
...,...,...
133060,923657,218
133061,923659,218
133075,941127,218
133083,947538,218


In [12]:
metrics_2_22_11_15__23_02_15.to_csv('metrics_2_22_11_15__23_02_15.csv')

In [13]:
del(metrics_2_22_11_15__23_02_15)

## 2022-12-01 - 2023-02-31

In [14]:
def get_core(store):
    query  = client.execute("""
    WITH
    models AS 
    (
    SELECT item AS line_item_id,
           name_ru AS new_class
    FROM assortment_ods.rms_product_attribute
    WHERE attribute = 'model_modelID'
    ), 
    --- 
    core_for_second AS 
    (
    SELECT store_id,
           line_item_id,
           ROUND(stddevPop(toInt128(col)),2) AS pvariance_delta_sku
    FROM 
        (SELECT sale_date,
                store_id,
                receipt_surrogate_id,
                line_item_id ,
                (sale_date - neighbor(sale_date,1,toDate('2001-01-01'))) AS delta,
                CASE
                    WHEN delta > 365 THEN 'не число'
                    WHEN delta < 0 THEN 'не число'
                    ELSE toString(delta)
                END AS col
        FROM 
            (SELECT DISTINCT l.opened_date AS sale_date,
                             l.store_id,
                             l.receipt_surrogate_id,
                             l.line_item_id
            FROM assortment_ods.dds_receipt_lines_public l
            LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
            WHERE l.opened_date BETWEEN '2022-12-01' -- начало интервала
                                    AND '2023-02-31' -- конец интервала
              AND l.line_type IN ('Sales', 'pickedUp orders')
              AND length(l.line_item_id) <= 8
              AND l.store_id = {}
            ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date desc) a
            ORDER BY  line_item_id desc, store_id desc, sale_date desc) C
        WHERE col != 'не число'
        GROUP BY  store_id, line_item_id
    ), 
    --- 
    core_for_fourth AS 
    (
    SELECT l.line_item_id,
            l.opened_date,
            l.store_id,
            SUM(line_turnover) AS turnover
    FROM assortment_ods.dds_receipt_lines_public l
    LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
    WHERE l.opened_date BETWEEN '2022-12-01' -- начало интервала
                            AND '2023-02-31' -- конец интервала
      AND l.line_type IN ('Sales', 'pickedUp orders')
      AND length(l.line_item_id) <= 8
      AND l.store_id = {}
    GROUP BY  l.line_item_id, l.opened_date, l.store_id
    ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date DESC 
    ), 
    --- 
    second_metrics AS 
    (
    SELECT core_for_second.store_id,
           core_for_second.line_item_id,
           models.new_class,
           core_for_second.pvariance_delta_sku,
           quantile(0.05)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS low_border, 
           quantile(0.95)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS high_border,
           CASE
               WHEN core_for_second.pvariance_delta_sku >= high_border THEN 'upper'
               WHEN core_for_second.pvariance_delta_sku < low_border THEN 'lower'
               ELSE 'middle'
           END AS label,
           CASE
               WHEN label = 'upper' THEN 0
               ELSE 1
           END AS core
    FROM core_for_second
    LEFT JOIN models ON core_for_second.line_item_id = models.line_item_id
    ORDER BY  core_for_second.store_id asc, core_for_second.line_item_id asc, models.new_class DESC
    ), 
    --- 
    fourth_metrics AS 
    (
    SELECT store_id,
           line_item_id,
           new_class,
           sales_days,
           lowers,
           quantile(0.15)(sales_days) OVER (PARTITION by store_id, new_class) AS border_days_model,
           CASE
               WHEN sales_days <= border_days_model THEN 0
               WHEN lowers > 0 THEN 0
               ELSE 1
           END AS core 
    FROM
        (SELECT store_id,
                line_item_id,
                new_class,
                sales_days,
                SUM(is_lower) AS lowers 
         FROM
             (SELECT store_id,
                     line_item_id,
                     new_class,
                     opened_date,
                     day_TO_ratio_sku,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id, label_TO) AS opened_dates,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id) AS sales_days,
                     opened_dates / sales_days * 100 AS perc_sales,
                     CASE
                         WHEN label_TO = 'lower' AND perc_sales >= (100-15) THEN 1
                         ELSE 0
                     END AS is_lower 
              FROM
                  (SELECT store_id,
                          line_item_id,
                          new_class,
                          opened_date,
                          TO_day_sku/ TO_day AS day_TO_ratio_sku,
                          quantile(0.15)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS low_border,
                          quantile(0.85)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS high_border,
                          CASE
                              WHEN day_TO_ratio_sku >= high_border THEN 'upper'
                              WHEN day_TO_ratio_sku < low_border THEN 'lower'
                              ELSE 'middle'
                          END AS label_TO
                   FROM
                       (SELECT store_id,
                               line_item_id,
                               new_class,
                               opened_date,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date) AS TO_day,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, line_item_id) AS TO_day_sku,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, new_class) AS TO_day_class 
                        FROM
                            (SELECT core_for_fourth.store_id AS store_id,
                                    core_for_fourth.line_item_id AS line_item_id,
                                    models.new_class AS new_class,
                                    core_for_fourth.opened_date,
                                    SUM(turnover) AS turnover
                             FROM core_for_fourth
                             LEFT JOIN models ON core_for_fourth.line_item_id = models.line_item_id
                             GROUP BY  core_for_fourth.store_id, core_for_fourth.line_item_id, models.new_class, core_for_fourth.opened_date) a) 
                   b) 
              c)
         d
    GROUP BY  store_id, line_item_id, new_class, sales_days) f) 
    ---
    SELECT s.store_id,
           s.line_item_id,
           s.new_class,
           s.label,
           s.pvariance_delta_sku,
           s.high_border,
           f.sales_days,
           f.lowers,
           f.border_days_model,
           CASE
               WHEN s.core = 1 AND f.core = 1 THEN 2
               WHEN s.core = 1 AND f.core = 0 THEN 2
               WHEN s.pvariance_delta_sku <= s.high_border *2 AND f.core = 1 THEN 2
               ELSE s.core + f.core
           END AS c
    FROM second_metrics AS s 
    FULL OUTER JOIN fourth_metrics AS f ON s.store_id = f.store_id
                                       AND s.line_item_id = f.line_item_id
                                       AND s.new_class = f.new_class
    ;
    """.format(store,store))
    return pd.DataFrame(query, columns=['store_id',
                                        'line_item_id',
                                        'new_class',
                                        'label',
                                        'pvariance_delta_sku',
                                        'high_border',
                                        'sales_days',
                                        'lowers',
                                        'border_days_model',
                                        'core'])

In [15]:
#создаем пустую таблицу для записи 
metrics_3_22_12_01__23_02_31 = pd.DataFrame()

# запускаем цикл по всем магазинам из списка
for store in true_stores:
    metrics_3_22_12_01__23_02_31 = pd.concat([metrics_3_22_12_01__23_02_31, get_core(store).query('store_id != 0')], ignore_index=True)

In [16]:
metrics_3_22_12_01__23_02_31.groupby(['line_item_id'], as_index=False).core.sum().query('core == 218')

,line_item_id,core
103,10051768,218
104,10051784,218
105,10051805,218
106,10052162,218
107,10052197,218
...,...,...
133473,941127,218
133478,945156,218
133481,947538,218
133534,956083,218


In [17]:
metrics_3_22_12_01__23_02_31.to_csv('metrics_3_22_12_01__23_02_31.csv')

In [18]:
del(metrics_3_22_12_01__23_02_31)

## 2022-12-15 - 2023-03-15

In [19]:
def get_core(store):
    query  = client.execute("""
    WITH
    models AS 
    (
    SELECT item AS line_item_id,
           name_ru AS new_class
    FROM assortment_ods.rms_product_attribute
    WHERE attribute = 'model_modelID'
    ), 
    --- 
    core_for_second AS 
    (
    SELECT store_id,
           line_item_id,
           ROUND(stddevPop(toInt128(col)),2) AS pvariance_delta_sku
    FROM 
        (SELECT sale_date,
                store_id,
                receipt_surrogate_id,
                line_item_id ,
                (sale_date - neighbor(sale_date,1,toDate('2001-01-01'))) AS delta,
                CASE
                    WHEN delta > 365 THEN 'не число'
                    WHEN delta < 0 THEN 'не число'
                    ELSE toString(delta)
                END AS col
        FROM 
            (SELECT DISTINCT l.opened_date AS sale_date,
                             l.store_id,
                             l.receipt_surrogate_id,
                             l.line_item_id
            FROM assortment_ods.dds_receipt_lines_public l
            LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
            WHERE l.opened_date BETWEEN '2022-12-15' -- начало интервала
                                    AND '2023-03-15' -- конец интервала
              AND l.line_type IN ('Sales', 'pickedUp orders')
              AND length(l.line_item_id) <= 8
              AND l.store_id = {}
            ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date desc) a
            ORDER BY  line_item_id desc, store_id desc, sale_date desc) C
        WHERE col != 'не число'
        GROUP BY  store_id, line_item_id
    ), 
    --- 
    core_for_fourth AS 
    (
    SELECT l.line_item_id,
            l.opened_date,
            l.store_id,
            SUM(line_turnover) AS turnover
    FROM assortment_ods.dds_receipt_lines_public l
    LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
    WHERE l.opened_date BETWEEN '2022-12-15' -- начало интервала
                            AND '2023-03-15' -- конец интервала
      AND l.line_type IN ('Sales', 'pickedUp orders')
      AND length(l.line_item_id) <= 8
      AND l.store_id = {}
    GROUP BY  l.line_item_id, l.opened_date, l.store_id
    ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date DESC 
    ), 
    --- 
    second_metrics AS 
    (
    SELECT core_for_second.store_id,
           core_for_second.line_item_id,
           models.new_class,
           core_for_second.pvariance_delta_sku,
           quantile(0.05)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS low_border, 
           quantile(0.95)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS high_border,
           CASE
               WHEN core_for_second.pvariance_delta_sku >= high_border THEN 'upper'
               WHEN core_for_second.pvariance_delta_sku < low_border THEN 'lower'
               ELSE 'middle'
           END AS label,
           CASE
               WHEN label = 'upper' THEN 0
               ELSE 1
           END AS core
    FROM core_for_second
    LEFT JOIN models ON core_for_second.line_item_id = models.line_item_id
    ORDER BY  core_for_second.store_id asc, core_for_second.line_item_id asc, models.new_class DESC
    ), 
    --- 
    fourth_metrics AS 
    (
    SELECT store_id,
           line_item_id,
           new_class,
           sales_days,
           lowers,
           quantile(0.15)(sales_days) OVER (PARTITION by store_id, new_class) AS border_days_model,
           CASE
               WHEN sales_days <= border_days_model THEN 0
               WHEN lowers > 0 THEN 0
               ELSE 1
           END AS core 
    FROM
        (SELECT store_id,
                line_item_id,
                new_class,
                sales_days,
                SUM(is_lower) AS lowers 
         FROM
             (SELECT store_id,
                     line_item_id,
                     new_class,
                     opened_date,
                     day_TO_ratio_sku,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id, label_TO) AS opened_dates,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id) AS sales_days,
                     opened_dates / sales_days * 100 AS perc_sales,
                     CASE
                         WHEN label_TO = 'lower' AND perc_sales >= (100-15) THEN 1
                         ELSE 0
                     END AS is_lower 
              FROM
                  (SELECT store_id,
                          line_item_id,
                          new_class,
                          opened_date,
                          TO_day_sku/ TO_day AS day_TO_ratio_sku,
                          quantile(0.15)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS low_border,
                          quantile(0.85)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS high_border,
                          CASE
                              WHEN day_TO_ratio_sku >= high_border THEN 'upper'
                              WHEN day_TO_ratio_sku < low_border THEN 'lower'
                              ELSE 'middle'
                          END AS label_TO
                   FROM
                       (SELECT store_id,
                               line_item_id,
                               new_class,
                               opened_date,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date) AS TO_day,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, line_item_id) AS TO_day_sku,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, new_class) AS TO_day_class 
                        FROM
                            (SELECT core_for_fourth.store_id AS store_id,
                                    core_for_fourth.line_item_id AS line_item_id,
                                    models.new_class AS new_class,
                                    core_for_fourth.opened_date,
                                    SUM(turnover) AS turnover
                             FROM core_for_fourth
                             LEFT JOIN models ON core_for_fourth.line_item_id = models.line_item_id
                             GROUP BY  core_for_fourth.store_id, core_for_fourth.line_item_id, models.new_class, core_for_fourth.opened_date) a) 
                   b) 
              c)
         d
    GROUP BY  store_id, line_item_id, new_class, sales_days) f) 
    ---
    SELECT s.store_id,
           s.line_item_id,
           s.new_class,
           s.label,
           s.pvariance_delta_sku,
           s.high_border,
           f.sales_days,
           f.lowers,
           f.border_days_model,
           CASE
               WHEN s.core = 1 AND f.core = 1 THEN 2
               WHEN s.core = 1 AND f.core = 0 THEN 2
               WHEN s.pvariance_delta_sku <= s.high_border *2 AND f.core = 1 THEN 2
               ELSE s.core + f.core
           END AS c
    FROM second_metrics AS s 
    FULL OUTER JOIN fourth_metrics AS f ON s.store_id = f.store_id
                                       AND s.line_item_id = f.line_item_id
                                       AND s.new_class = f.new_class
    ;
    """.format(store,store))
    return pd.DataFrame(query, columns=['store_id',
                                        'line_item_id',
                                        'new_class',
                                        'label',
                                        'pvariance_delta_sku',
                                        'high_border',
                                        'sales_days',
                                        'lowers',
                                        'border_days_model',
                                        'core'])

In [20]:
#создаем пустую таблицу для записи 
metrics_4_22_12_15__23_03_15 = pd.DataFrame()

# запускаем цикл по всем магазинам из списка
for store in true_stores:
    metrics_4_22_12_15__23_03_15 = pd.concat([metrics_4_22_12_15__23_03_15, get_core(store).query('store_id != 0')], ignore_index=True)

In [21]:
metrics_4_22_12_15__23_03_15.groupby(['line_item_id'], as_index=False).core.sum().query('core == 218')

,line_item_id,core
102,10051768,218
103,10051784,218
104,10051805,218
105,10052162,218
106,10052197,218
...,...,...
133556,923657,218
133557,923659,218
133571,941127,218
133576,945156,218


In [22]:
metrics_4_22_12_15__23_03_15.to_csv('metrics_4_22_12_15__23_03_15.csv')

In [23]:
del(metrics_4_22_12_15__23_03_15)

## 2023-01-01 - 2023-03-31

In [24]:
def get_core(store):
    query  = client.execute("""
    WITH
    models AS 
    (
    SELECT item AS line_item_id,
           name_ru AS new_class
    FROM assortment_ods.rms_product_attribute
    WHERE attribute = 'model_modelID'
    ), 
    --- 
    core_for_second AS 
    (
    SELECT store_id,
           line_item_id,
           ROUND(stddevPop(toInt128(col)),2) AS pvariance_delta_sku
    FROM 
        (SELECT sale_date,
                store_id,
                receipt_surrogate_id,
                line_item_id ,
                (sale_date - neighbor(sale_date,1,toDate('2001-01-01'))) AS delta,
                CASE
                    WHEN delta > 365 THEN 'не число'
                    WHEN delta < 0 THEN 'не число'
                    ELSE toString(delta)
                END AS col
        FROM 
            (SELECT DISTINCT l.opened_date AS sale_date,
                             l.store_id,
                             l.receipt_surrogate_id,
                             l.line_item_id
            FROM assortment_ods.dds_receipt_lines_public l
            LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
            WHERE l.opened_date BETWEEN '2023-01-01' -- начало интервала
                                    AND '2023-03-31' -- конец интервала
              AND l.line_type IN ('Sales', 'pickedUp orders')
              AND length(l.line_item_id) <= 8
              AND l.store_id = {}
            ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date desc) a
            ORDER BY  line_item_id desc, store_id desc, sale_date desc) C
        WHERE col != 'не число'
        GROUP BY  store_id, line_item_id
    ), 
    --- 
    core_for_fourth AS 
    (
    SELECT l.line_item_id,
            l.opened_date,
            l.store_id,
            SUM(line_turnover) AS turnover
    FROM assortment_ods.dds_receipt_lines_public l
    LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
    WHERE l.opened_date BETWEEN '2023-01-01' -- начало интервала
                            AND '2023-03-31' -- конец интервала
      AND l.line_type IN ('Sales', 'pickedUp orders')
      AND length(l.line_item_id) <= 8
      AND l.store_id = {}
    GROUP BY  l.line_item_id, l.opened_date, l.store_id
    ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date DESC 
    ), 
    --- 
    second_metrics AS 
    (
    SELECT core_for_second.store_id,
           core_for_second.line_item_id,
           models.new_class,
           core_for_second.pvariance_delta_sku,
           quantile(0.05)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS low_border, 
           quantile(0.95)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS high_border,
           CASE
               WHEN core_for_second.pvariance_delta_sku >= high_border THEN 'upper'
               WHEN core_for_second.pvariance_delta_sku < low_border THEN 'lower'
               ELSE 'middle'
           END AS label,
           CASE
               WHEN label = 'upper' THEN 0
               ELSE 1
           END AS core
    FROM core_for_second
    LEFT JOIN models ON core_for_second.line_item_id = models.line_item_id
    ORDER BY  core_for_second.store_id asc, core_for_second.line_item_id asc, models.new_class DESC
    ), 
    --- 
    fourth_metrics AS 
    (
    SELECT store_id,
           line_item_id,
           new_class,
           sales_days,
           lowers,
           quantile(0.15)(sales_days) OVER (PARTITION by store_id, new_class) AS border_days_model,
           CASE
               WHEN sales_days <= border_days_model THEN 0
               WHEN lowers > 0 THEN 0
               ELSE 1
           END AS core 
    FROM
        (SELECT store_id,
                line_item_id,
                new_class,
                sales_days,
                SUM(is_lower) AS lowers 
         FROM
             (SELECT store_id,
                     line_item_id,
                     new_class,
                     opened_date,
                     day_TO_ratio_sku,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id, label_TO) AS opened_dates,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id) AS sales_days,
                     opened_dates / sales_days * 100 AS perc_sales,
                     CASE
                         WHEN label_TO = 'lower' AND perc_sales >= (100-15) THEN 1
                         ELSE 0
                     END AS is_lower 
              FROM
                  (SELECT store_id,
                          line_item_id,
                          new_class,
                          opened_date,
                          TO_day_sku/ TO_day AS day_TO_ratio_sku,
                          quantile(0.15)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS low_border,
                          quantile(0.85)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS high_border,
                          CASE
                              WHEN day_TO_ratio_sku >= high_border THEN 'upper'
                              WHEN day_TO_ratio_sku < low_border THEN 'lower'
                              ELSE 'middle'
                          END AS label_TO
                   FROM
                       (SELECT store_id,
                               line_item_id,
                               new_class,
                               opened_date,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date) AS TO_day,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, line_item_id) AS TO_day_sku,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, new_class) AS TO_day_class 
                        FROM
                            (SELECT core_for_fourth.store_id AS store_id,
                                    core_for_fourth.line_item_id AS line_item_id,
                                    models.new_class AS new_class,
                                    core_for_fourth.opened_date,
                                    SUM(turnover) AS turnover
                             FROM core_for_fourth
                             LEFT JOIN models ON core_for_fourth.line_item_id = models.line_item_id
                             GROUP BY  core_for_fourth.store_id, core_for_fourth.line_item_id, models.new_class, core_for_fourth.opened_date) a) 
                   b) 
              c)
         d
    GROUP BY  store_id, line_item_id, new_class, sales_days) f) 
    ---
    SELECT s.store_id,
           s.line_item_id,
           s.new_class,
           s.label,
           s.pvariance_delta_sku,
           s.high_border,
           f.sales_days,
           f.lowers,
           f.border_days_model,
           CASE
               WHEN s.core = 1 AND f.core = 1 THEN 2
               WHEN s.core = 1 AND f.core = 0 THEN 2
               WHEN s.pvariance_delta_sku <= s.high_border *2 AND f.core = 1 THEN 2
               ELSE s.core + f.core
           END AS c
    FROM second_metrics AS s 
    FULL OUTER JOIN fourth_metrics AS f ON s.store_id = f.store_id
                                       AND s.line_item_id = f.line_item_id
                                       AND s.new_class = f.new_class
    ;
    """.format(store,store))
    return pd.DataFrame(query, columns=['store_id',
                                        'line_item_id',
                                        'new_class',
                                        'label',
                                        'pvariance_delta_sku',
                                        'high_border',
                                        'sales_days',
                                        'lowers',
                                        'border_days_model',
                                        'core'])

In [25]:
#создаем пустую таблицу для записи 
metrics_5_23_01_01__23_03_31 = pd.DataFrame()

# запускаем цикл по всем магазинам из списка
for store in true_stores:
    metrics_5_23_01_01__23_03_31 = pd.concat([metrics_5_23_01_01__23_03_31, get_core(store).query('store_id != 0')], ignore_index=True)

In [26]:
metrics_5_23_01_01__23_03_31.groupby(['line_item_id'], as_index=False).core.sum().query('core == 218')

,line_item_id,core
104,10051768,218
105,10051784,218
106,10051805,218
107,10052162,218
108,10052197,218
...,...,...
133138,923653,218
133140,923657,218
133141,923659,218
133155,941127,218


In [27]:
metrics_5_23_01_01__23_03_31.to_csv('metrics_5_23_01_01__23_03_31.csv')

In [28]:
del(metrics_5_23_01_01__23_03_31)

## 2023-01-15 - 2023-04-15

In [29]:
def get_core(store):
    query  = client.execute("""
    WITH
    models AS 
    (
    SELECT item AS line_item_id,
           name_ru AS new_class
    FROM assortment_ods.rms_product_attribute
    WHERE attribute = 'model_modelID'
    ), 
    --- 
    core_for_second AS 
    (
    SELECT store_id,
           line_item_id,
           ROUND(stddevPop(toInt128(col)),2) AS pvariance_delta_sku
    FROM 
        (SELECT sale_date,
                store_id,
                receipt_surrogate_id,
                line_item_id ,
                (sale_date - neighbor(sale_date,1,toDate('2001-01-01'))) AS delta,
                CASE
                    WHEN delta > 365 THEN 'не число'
                    WHEN delta < 0 THEN 'не число'
                    ELSE toString(delta)
                END AS col
        FROM 
            (SELECT DISTINCT l.opened_date AS sale_date,
                             l.store_id,
                             l.receipt_surrogate_id,
                             l.line_item_id
            FROM assortment_ods.dds_receipt_lines_public l
            LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
            WHERE l.opened_date BETWEEN '2023-01-15' -- начало интервала
                                    AND '2023-04-15' -- конец интервала
              AND l.line_type IN ('Sales', 'pickedUp orders')
              AND length(l.line_item_id) <= 8
              AND l.store_id = {}
            ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date desc) a
            ORDER BY  line_item_id desc, store_id desc, sale_date desc) C
        WHERE col != 'не число'
        GROUP BY  store_id, line_item_id
    ), 
    --- 
    core_for_fourth AS 
    (
    SELECT l.line_item_id,
            l.opened_date,
            l.store_id,
            SUM(line_turnover) AS turnover
    FROM assortment_ods.dds_receipt_lines_public l
    LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
    WHERE l.opened_date BETWEEN '2023-01-15' -- начало интервала
                            AND '2023-04-15' -- конец интервала
      AND l.line_type IN ('Sales', 'pickedUp orders')
      AND length(l.line_item_id) <= 8
      AND l.store_id = {}
    GROUP BY  l.line_item_id, l.opened_date, l.store_id
    ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date DESC 
    ), 
    --- 
    second_metrics AS 
    (
    SELECT core_for_second.store_id,
           core_for_second.line_item_id,
           models.new_class,
           core_for_second.pvariance_delta_sku,
           quantile(0.05)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS low_border, 
           quantile(0.95)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS high_border,
           CASE
               WHEN core_for_second.pvariance_delta_sku >= high_border THEN 'upper'
               WHEN core_for_second.pvariance_delta_sku < low_border THEN 'lower'
               ELSE 'middle'
           END AS label,
           CASE
               WHEN label = 'upper' THEN 0
               ELSE 1
           END AS core
    FROM core_for_second
    LEFT JOIN models ON core_for_second.line_item_id = models.line_item_id
    ORDER BY  core_for_second.store_id asc, core_for_second.line_item_id asc, models.new_class DESC
    ), 
    --- 
    fourth_metrics AS 
    (
    SELECT store_id,
           line_item_id,
           new_class,
           sales_days,
           lowers,
           quantile(0.15)(sales_days) OVER (PARTITION by store_id, new_class) AS border_days_model,
           CASE
               WHEN sales_days <= border_days_model THEN 0
               WHEN lowers > 0 THEN 0
               ELSE 1
           END AS core 
    FROM
        (SELECT store_id,
                line_item_id,
                new_class,
                sales_days,
                SUM(is_lower) AS lowers 
         FROM
             (SELECT store_id,
                     line_item_id,
                     new_class,
                     opened_date,
                     day_TO_ratio_sku,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id, label_TO) AS opened_dates,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id) AS sales_days,
                     opened_dates / sales_days * 100 AS perc_sales,
                     CASE
                         WHEN label_TO = 'lower' AND perc_sales >= (100-15) THEN 1
                         ELSE 0
                     END AS is_lower 
              FROM
                  (SELECT store_id,
                          line_item_id,
                          new_class,
                          opened_date,
                          TO_day_sku/ TO_day AS day_TO_ratio_sku,
                          quantile(0.15)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS low_border,
                          quantile(0.85)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS high_border,
                          CASE
                              WHEN day_TO_ratio_sku >= high_border THEN 'upper'
                              WHEN day_TO_ratio_sku < low_border THEN 'lower'
                              ELSE 'middle'
                          END AS label_TO
                   FROM
                       (SELECT store_id,
                               line_item_id,
                               new_class,
                               opened_date,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date) AS TO_day,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, line_item_id) AS TO_day_sku,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, new_class) AS TO_day_class 
                        FROM
                            (SELECT core_for_fourth.store_id AS store_id,
                                    core_for_fourth.line_item_id AS line_item_id,
                                    models.new_class AS new_class,
                                    core_for_fourth.opened_date,
                                    SUM(turnover) AS turnover
                             FROM core_for_fourth
                             LEFT JOIN models ON core_for_fourth.line_item_id = models.line_item_id
                             GROUP BY  core_for_fourth.store_id, core_for_fourth.line_item_id, models.new_class, core_for_fourth.opened_date) a) 
                   b) 
              c)
         d
    GROUP BY  store_id, line_item_id, new_class, sales_days) f) 
    ---
    SELECT s.store_id,
           s.line_item_id,
           s.new_class,
           s.label,
           s.pvariance_delta_sku,
           s.high_border,
           f.sales_days,
           f.lowers,
           f.border_days_model,
           CASE
               WHEN s.core = 1 AND f.core = 1 THEN 2
               WHEN s.core = 1 AND f.core = 0 THEN 2
               WHEN s.pvariance_delta_sku <= s.high_border *2 AND f.core = 1 THEN 2
               ELSE s.core + f.core
           END AS c
    FROM second_metrics AS s 
    FULL OUTER JOIN fourth_metrics AS f ON s.store_id = f.store_id
                                       AND s.line_item_id = f.line_item_id
                                       AND s.new_class = f.new_class
    ;
    """.format(store,store))
    return pd.DataFrame(query, columns=['store_id',
                                        'line_item_id',
                                        'new_class',
                                        'label',
                                        'pvariance_delta_sku',
                                        'high_border',
                                        'sales_days',
                                        'lowers',
                                        'border_days_model',
                                        'core'])

In [30]:
#создаем пустую таблицу для записи 
metrics_6_23_01_15__23_04_15 = pd.DataFrame()

# запускаем цикл по всем магазинам из списка
for store in true_stores:
    metrics_6_23_01_15__23_04_15 = pd.concat([metrics_6_23_01_15__23_04_15, get_core(store).query('store_id != 0')], ignore_index=True)

In [31]:
metrics_6_23_01_15__23_04_15.groupby(['line_item_id'], as_index=False).core.sum().query('core == 218')

,line_item_id,core
104,10051768,218
105,10051784,218
106,10051805,218
107,10052162,218
108,10052197,218
...,...,...
134189,923651,218
134190,923653,218
134192,923657,218
134193,923659,218


In [32]:
metrics_6_23_01_15__23_04_15.to_csv('metrics_6_23_01_15__23_04_15.csv')

In [33]:
del(metrics_6_23_01_15__23_04_15)

## 2023-02-01 - 2023-04-31

In [34]:
def get_core(store):
    query  = client.execute("""
    WITH
    models AS 
    (
    SELECT item AS line_item_id,
           name_ru AS new_class
    FROM assortment_ods.rms_product_attribute
    WHERE attribute = 'model_modelID'
    ), 
    --- 
    core_for_second AS 
    (
    SELECT store_id,
           line_item_id,
           ROUND(stddevPop(toInt128(col)),2) AS pvariance_delta_sku
    FROM 
        (SELECT sale_date,
                store_id,
                receipt_surrogate_id,
                line_item_id ,
                (sale_date - neighbor(sale_date,1,toDate('2001-01-01'))) AS delta,
                CASE
                    WHEN delta > 365 THEN 'не число'
                    WHEN delta < 0 THEN 'не число'
                    ELSE toString(delta)
                END AS col
        FROM 
            (SELECT DISTINCT l.opened_date AS sale_date,
                             l.store_id,
                             l.receipt_surrogate_id,
                             l.line_item_id
            FROM assortment_ods.dds_receipt_lines_public l
            LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
            WHERE l.opened_date BETWEEN '2023-02-01' -- начало интервала
                                    AND '2023-04-31' -- конец интервала
              AND l.line_type IN ('Sales', 'pickedUp orders')
              AND length(l.line_item_id) <= 8
              AND l.store_id = {}
            ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date desc) a
            ORDER BY  line_item_id desc, store_id desc, sale_date desc) C
        WHERE col != 'не число'
        GROUP BY  store_id, line_item_id
    ), 
    --- 
    core_for_fourth AS 
    (
    SELECT l.line_item_id,
            l.opened_date,
            l.store_id,
            SUM(line_turnover) AS turnover
    FROM assortment_ods.dds_receipt_lines_public l
    LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
    WHERE l.opened_date BETWEEN '2023-02-01' -- начало интервала
                            AND '2023-04-31' -- конец интервала
      AND l.line_type IN ('Sales', 'pickedUp orders')
      AND length(l.line_item_id) <= 8
      AND l.store_id = {}
    GROUP BY  l.line_item_id, l.opened_date, l.store_id
    ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date DESC 
    ), 
    --- 
    second_metrics AS 
    (
    SELECT core_for_second.store_id,
           core_for_second.line_item_id,
           models.new_class,
           core_for_second.pvariance_delta_sku,
           quantile(0.05)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS low_border, 
           quantile(0.95)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS high_border,
           CASE
               WHEN core_for_second.pvariance_delta_sku >= high_border THEN 'upper'
               WHEN core_for_second.pvariance_delta_sku < low_border THEN 'lower'
               ELSE 'middle'
           END AS label,
           CASE
               WHEN label = 'upper' THEN 0
               ELSE 1
           END AS core
    FROM core_for_second
    LEFT JOIN models ON core_for_second.line_item_id = models.line_item_id
    ORDER BY  core_for_second.store_id asc, core_for_second.line_item_id asc, models.new_class DESC
    ), 
    --- 
    fourth_metrics AS 
    (
    SELECT store_id,
           line_item_id,
           new_class,
           sales_days,
           lowers,
           quantile(0.15)(sales_days) OVER (PARTITION by store_id, new_class) AS border_days_model,
           CASE
               WHEN sales_days <= border_days_model THEN 0
               WHEN lowers > 0 THEN 0
               ELSE 1
           END AS core 
    FROM
        (SELECT store_id,
                line_item_id,
                new_class,
                sales_days,
                SUM(is_lower) AS lowers 
         FROM
             (SELECT store_id,
                     line_item_id,
                     new_class,
                     opened_date,
                     day_TO_ratio_sku,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id, label_TO) AS opened_dates,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id) AS sales_days,
                     opened_dates / sales_days * 100 AS perc_sales,
                     CASE
                         WHEN label_TO = 'lower' AND perc_sales >= (100-15) THEN 1
                         ELSE 0
                     END AS is_lower 
              FROM
                  (SELECT store_id,
                          line_item_id,
                          new_class,
                          opened_date,
                          TO_day_sku/ TO_day AS day_TO_ratio_sku,
                          quantile(0.15)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS low_border,
                          quantile(0.85)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS high_border,
                          CASE
                              WHEN day_TO_ratio_sku >= high_border THEN 'upper'
                              WHEN day_TO_ratio_sku < low_border THEN 'lower'
                              ELSE 'middle'
                          END AS label_TO
                   FROM
                       (SELECT store_id,
                               line_item_id,
                               new_class,
                               opened_date,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date) AS TO_day,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, line_item_id) AS TO_day_sku,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, new_class) AS TO_day_class 
                        FROM
                            (SELECT core_for_fourth.store_id AS store_id,
                                    core_for_fourth.line_item_id AS line_item_id,
                                    models.new_class AS new_class,
                                    core_for_fourth.opened_date,
                                    SUM(turnover) AS turnover
                             FROM core_for_fourth
                             LEFT JOIN models ON core_for_fourth.line_item_id = models.line_item_id
                             GROUP BY  core_for_fourth.store_id, core_for_fourth.line_item_id, models.new_class, core_for_fourth.opened_date) a) 
                   b) 
              c)
         d
    GROUP BY  store_id, line_item_id, new_class, sales_days) f) 
    ---
    SELECT s.store_id,
           s.line_item_id,
           s.new_class,
           s.label,
           s.pvariance_delta_sku,
           s.high_border,
           f.sales_days,
           f.lowers,
           f.border_days_model,
           CASE
               WHEN s.core = 1 AND f.core = 1 THEN 2
               WHEN s.core = 1 AND f.core = 0 THEN 2
               WHEN s.pvariance_delta_sku <= s.high_border *2 AND f.core = 1 THEN 2
               ELSE s.core + f.core
           END AS c
    FROM second_metrics AS s 
    FULL OUTER JOIN fourth_metrics AS f ON s.store_id = f.store_id
                                       AND s.line_item_id = f.line_item_id
                                       AND s.new_class = f.new_class
    ;
    """.format(store,store))
    return pd.DataFrame(query, columns=['store_id',
                                        'line_item_id',
                                        'new_class',
                                        'label',
                                        'pvariance_delta_sku',
                                        'high_border',
                                        'sales_days',
                                        'lowers',
                                        'border_days_model',
                                        'core'])

In [35]:
#создаем пустую таблицу для записи 
metrics_7_23_02_01__23_04_31 = pd.DataFrame()

# запускаем цикл по всем магазинам из списка
for store in true_stores:
    metrics_7_23_02_01__23_04_31 = pd.concat([metrics_7_23_02_01__23_04_31, get_core(store).query('store_id != 0')], ignore_index=True)

In [36]:
metrics_7_23_02_01__23_04_31.groupby(['line_item_id'], as_index=False).core.sum().query('core == 218')

,line_item_id,core
105,10051768,218
106,10051784,218
107,10051805,218
108,10052162,218
109,10052197,218
...,...,...
136330,923576,218
136332,923651,218
136333,923653,218
136335,923657,218


In [37]:
metrics_7_23_02_01__23_04_31.to_csv('metrics_7_23_02_01__23_04_31.csv')

In [38]:
del(metrics_7_23_02_01__23_04_31)

## 2023-02-15 - 2023-05-15

In [39]:
def get_core(store):
    query  = client.execute("""
    WITH
    models AS 
    (
    SELECT item AS line_item_id,
           name_ru AS new_class
    FROM assortment_ods.rms_product_attribute
    WHERE attribute = 'model_modelID'
    ), 
    --- 
    core_for_second AS 
    (
    SELECT store_id,
           line_item_id,
           ROUND(stddevPop(toInt128(col)),2) AS pvariance_delta_sku
    FROM 
        (SELECT sale_date,
                store_id,
                receipt_surrogate_id,
                line_item_id ,
                (sale_date - neighbor(sale_date,1,toDate('2001-01-01'))) AS delta,
                CASE
                    WHEN delta > 365 THEN 'не число'
                    WHEN delta < 0 THEN 'не число'
                    ELSE toString(delta)
                END AS col
        FROM 
            (SELECT DISTINCT l.opened_date AS sale_date,
                             l.store_id,
                             l.receipt_surrogate_id,
                             l.line_item_id
            FROM assortment_ods.dds_receipt_lines_public l
            LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
            WHERE l.opened_date BETWEEN '2023-02-15' -- начало интервала
                                    AND '2023-05-15' -- конец интервала
              AND l.line_type IN ('Sales', 'pickedUp orders')
              AND length(l.line_item_id) <= 8
              AND l.store_id = {}
            ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date desc) a
            ORDER BY  line_item_id desc, store_id desc, sale_date desc) C
        WHERE col != 'не число'
        GROUP BY  store_id, line_item_id
    ), 
    --- 
    core_for_fourth AS 
    (
    SELECT l.line_item_id,
            l.opened_date,
            l.store_id,
            SUM(line_turnover) AS turnover
    FROM assortment_ods.dds_receipt_lines_public l
    LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
    WHERE l.opened_date BETWEEN '2023-02-15' -- начало интервала
                            AND '2023-05-15' -- конец интервала
      AND l.line_type IN ('Sales', 'pickedUp orders')
      AND length(l.line_item_id) <= 8
      AND l.store_id = {}
    GROUP BY  l.line_item_id, l.opened_date, l.store_id
    ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date DESC 
    ), 
    --- 
    second_metrics AS 
    (
    SELECT core_for_second.store_id,
           core_for_second.line_item_id,
           models.new_class,
           core_for_second.pvariance_delta_sku,
           quantile(0.05)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS low_border, 
           quantile(0.95)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS high_border,
           CASE
               WHEN core_for_second.pvariance_delta_sku >= high_border THEN 'upper'
               WHEN core_for_second.pvariance_delta_sku < low_border THEN 'lower'
               ELSE 'middle'
           END AS label,
           CASE
               WHEN label = 'upper' THEN 0
               ELSE 1
           END AS core
    FROM core_for_second
    LEFT JOIN models ON core_for_second.line_item_id = models.line_item_id
    ORDER BY  core_for_second.store_id asc, core_for_second.line_item_id asc, models.new_class DESC
    ), 
    --- 
    fourth_metrics AS 
    (
    SELECT store_id,
           line_item_id,
           new_class,
           sales_days,
           lowers,
           quantile(0.15)(sales_days) OVER (PARTITION by store_id, new_class) AS border_days_model,
           CASE
               WHEN sales_days <= border_days_model THEN 0
               WHEN lowers > 0 THEN 0
               ELSE 1
           END AS core 
    FROM
        (SELECT store_id,
                line_item_id,
                new_class,
                sales_days,
                SUM(is_lower) AS lowers 
         FROM
             (SELECT store_id,
                     line_item_id,
                     new_class,
                     opened_date,
                     day_TO_ratio_sku,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id, label_TO) AS opened_dates,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id) AS sales_days,
                     opened_dates / sales_days * 100 AS perc_sales,
                     CASE
                         WHEN label_TO = 'lower' AND perc_sales >= (100-15) THEN 1
                         ELSE 0
                     END AS is_lower 
              FROM
                  (SELECT store_id,
                          line_item_id,
                          new_class,
                          opened_date,
                          TO_day_sku/ TO_day AS day_TO_ratio_sku,
                          quantile(0.15)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS low_border,
                          quantile(0.85)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS high_border,
                          CASE
                              WHEN day_TO_ratio_sku >= high_border THEN 'upper'
                              WHEN day_TO_ratio_sku < low_border THEN 'lower'
                              ELSE 'middle'
                          END AS label_TO
                   FROM
                       (SELECT store_id,
                               line_item_id,
                               new_class,
                               opened_date,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date) AS TO_day,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, line_item_id) AS TO_day_sku,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, new_class) AS TO_day_class 
                        FROM
                            (SELECT core_for_fourth.store_id AS store_id,
                                    core_for_fourth.line_item_id AS line_item_id,
                                    models.new_class AS new_class,
                                    core_for_fourth.opened_date,
                                    SUM(turnover) AS turnover
                             FROM core_for_fourth
                             LEFT JOIN models ON core_for_fourth.line_item_id = models.line_item_id
                             GROUP BY  core_for_fourth.store_id, core_for_fourth.line_item_id, models.new_class, core_for_fourth.opened_date) a) 
                   b) 
              c)
         d
    GROUP BY  store_id, line_item_id, new_class, sales_days) f) 
    ---
    SELECT s.store_id,
           s.line_item_id,
           s.new_class,
           s.label,
           s.pvariance_delta_sku,
           s.high_border,
           f.sales_days,
           f.lowers,
           f.border_days_model,
           CASE
               WHEN s.core = 1 AND f.core = 1 THEN 2
               WHEN s.core = 1 AND f.core = 0 THEN 2
               WHEN s.pvariance_delta_sku <= s.high_border *2 AND f.core = 1 THEN 2
               ELSE s.core + f.core
           END AS c
    FROM second_metrics AS s 
    FULL OUTER JOIN fourth_metrics AS f ON s.store_id = f.store_id
                                       AND s.line_item_id = f.line_item_id
                                       AND s.new_class = f.new_class
    ;
    """.format(store,store))
    return pd.DataFrame(query, columns=['store_id',
                                        'line_item_id',
                                        'new_class',
                                        'label',
                                        'pvariance_delta_sku',
                                        'high_border',
                                        'sales_days',
                                        'lowers',
                                        'border_days_model',
                                        'core'])

In [40]:
#создаем пустую таблицу для записи 
metrics_8_23_02_15__23_05_15 = pd.DataFrame()

# запускаем цикл по всем магазинам из списка
for store in true_stores:
    metrics_8_23_02_15__23_05_15 = pd.concat([metrics_8_23_02_15__23_05_15, get_core(store).query('store_id != 0')], ignore_index=True)

In [41]:
metrics_8_23_02_15__23_05_15.groupby(['line_item_id'], as_index=False).core.sum().query('core == 218')

,line_item_id,core
103,10051768,218
104,10051784,218
105,10051805,218
106,10052162,218
107,10052197,218
...,...,...
138143,923651,218
138144,923653,218
138146,923657,218
138147,923659,218


In [42]:
metrics_8_23_02_15__23_05_15.to_csv('metrics_8_23_02_15__23_05_15.csv')

In [43]:
del(metrics_8_23_02_15__23_05_15)

## 2023-03-01 - 2023-05-31

In [44]:
def get_core(store):
    query  = client.execute("""
    WITH
    models AS 
    (
    SELECT item AS line_item_id,
           name_ru AS new_class
    FROM assortment_ods.rms_product_attribute
    WHERE attribute = 'model_modelID'
    ), 
    --- 
    core_for_second AS 
    (
    SELECT store_id,
           line_item_id,
           ROUND(stddevPop(toInt128(col)),2) AS pvariance_delta_sku
    FROM 
        (SELECT sale_date,
                store_id,
                receipt_surrogate_id,
                line_item_id ,
                (sale_date - neighbor(sale_date,1,toDate('2001-01-01'))) AS delta,
                CASE
                    WHEN delta > 365 THEN 'не число'
                    WHEN delta < 0 THEN 'не число'
                    ELSE toString(delta)
                END AS col
        FROM 
            (SELECT DISTINCT l.opened_date AS sale_date,
                             l.store_id,
                             l.receipt_surrogate_id,
                             l.line_item_id
            FROM assortment_ods.dds_receipt_lines_public l
            LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
            WHERE l.opened_date BETWEEN '2023-03-01' -- начало интервала
                                    AND '2023-05-31' -- конец интервала
              AND l.line_type IN ('Sales', 'pickedUp orders')
              AND length(l.line_item_id) <= 8
              AND l.store_id = {}
            ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date desc) a
            ORDER BY  line_item_id desc, store_id desc, sale_date desc) C
        WHERE col != 'не число'
        GROUP BY  store_id, line_item_id
    ), 
    --- 
    core_for_fourth AS 
    (
    SELECT l.line_item_id,
            l.opened_date,
            l.store_id,
            SUM(line_turnover) AS turnover
    FROM assortment_ods.dds_receipt_lines_public l
    LEFT JOIN assortment_ods.rms_ods_item_master r ON l.line_item_id = r.item
    WHERE l.opened_date BETWEEN '2023-03-01' -- начало интервала
                            AND '2023-05-31' -- конец интервала
      AND l.line_type IN ('Sales', 'pickedUp orders')
      AND length(l.line_item_id) <= 8
      AND l.store_id = {}
    GROUP BY  l.line_item_id, l.opened_date, l.store_id
    ORDER BY  l.line_item_id desc, l.opened_date desc, l.store_id desc, l.opened_date DESC 
    ), 
    --- 
    second_metrics AS 
    (
    SELECT core_for_second.store_id,
           core_for_second.line_item_id,
           models.new_class,
           core_for_second.pvariance_delta_sku,
           quantile(0.05)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS low_border, 
           quantile(0.95)(core_for_second.pvariance_delta_sku) OVER (PARTITION by core_for_second.store_id, models.new_class) AS high_border,
           CASE
               WHEN core_for_second.pvariance_delta_sku >= high_border THEN 'upper'
               WHEN core_for_second.pvariance_delta_sku < low_border THEN 'lower'
               ELSE 'middle'
           END AS label,
           CASE
               WHEN label = 'upper' THEN 0
               ELSE 1
           END AS core
    FROM core_for_second
    LEFT JOIN models ON core_for_second.line_item_id = models.line_item_id
    ORDER BY  core_for_second.store_id asc, core_for_second.line_item_id asc, models.new_class DESC
    ), 
    --- 
    fourth_metrics AS 
    (
    SELECT store_id,
           line_item_id,
           new_class,
           sales_days,
           lowers,
           quantile(0.15)(sales_days) OVER (PARTITION by store_id, new_class) AS border_days_model,
           CASE
               WHEN sales_days <= border_days_model THEN 0
               WHEN lowers > 0 THEN 0
               ELSE 1
           END AS core 
    FROM
        (SELECT store_id,
                line_item_id,
                new_class,
                sales_days,
                SUM(is_lower) AS lowers 
         FROM
             (SELECT store_id,
                     line_item_id,
                     new_class,
                     opened_date,
                     day_TO_ratio_sku,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id, label_TO) AS opened_dates,
                     COUNT(DISTINCT opened_date) OVER (PARTITION by store_id, line_item_id) AS sales_days,
                     opened_dates / sales_days * 100 AS perc_sales,
                     CASE
                         WHEN label_TO = 'lower' AND perc_sales >= (100-15) THEN 1
                         ELSE 0
                     END AS is_lower 
              FROM
                  (SELECT store_id,
                          line_item_id,
                          new_class,
                          opened_date,
                          TO_day_sku/ TO_day AS day_TO_ratio_sku,
                          quantile(0.15)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS low_border,
                          quantile(0.85)(day_TO_ratio_sku) OVER (PARTITION by store_id, new_class) AS high_border,
                          CASE
                              WHEN day_TO_ratio_sku >= high_border THEN 'upper'
                              WHEN day_TO_ratio_sku < low_border THEN 'lower'
                              ELSE 'middle'
                          END AS label_TO
                   FROM
                       (SELECT store_id,
                               line_item_id,
                               new_class,
                               opened_date,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date) AS TO_day,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, line_item_id) AS TO_day_sku,
                               SUM(turnover) OVER (PARTITION by store_id, opened_date, new_class) AS TO_day_class 
                        FROM
                            (SELECT core_for_fourth.store_id AS store_id,
                                    core_for_fourth.line_item_id AS line_item_id,
                                    models.new_class AS new_class,
                                    core_for_fourth.opened_date,
                                    SUM(turnover) AS turnover
                             FROM core_for_fourth
                             LEFT JOIN models ON core_for_fourth.line_item_id = models.line_item_id
                             GROUP BY  core_for_fourth.store_id, core_for_fourth.line_item_id, models.new_class, core_for_fourth.opened_date) a) 
                   b) 
              c)
         d
    GROUP BY  store_id, line_item_id, new_class, sales_days) f) 
    ---
    SELECT s.store_id,
           s.line_item_id,
           s.new_class,
           s.label,
           s.pvariance_delta_sku,
           s.high_border,
           f.sales_days,
           f.lowers,
           f.border_days_model,
           CASE
               WHEN s.core = 1 AND f.core = 1 THEN 2
               WHEN s.core = 1 AND f.core = 0 THEN 2
               WHEN s.pvariance_delta_sku <= s.high_border *2 AND f.core = 1 THEN 2
               ELSE s.core + f.core
           END AS c
    FROM second_metrics AS s 
    FULL OUTER JOIN fourth_metrics AS f ON s.store_id = f.store_id
                                       AND s.line_item_id = f.line_item_id
                                       AND s.new_class = f.new_class
    ;
    """.format(store,store))
    return pd.DataFrame(query, columns=['store_id',
                                        'line_item_id',
                                        'new_class',
                                        'label',
                                        'pvariance_delta_sku',
                                        'high_border',
                                        'sales_days',
                                        'lowers',
                                        'border_days_model',
                                        'core'])

In [45]:
#создаем пустую таблицу для записи 
metrics_9_23_03_01__23_05_31 = pd.DataFrame()

# запускаем цикл по всем магазинам из списка
for store in true_stores:
    metrics_9_23_03_01__23_05_31 = pd.concat([metrics_9_23_03_01__23_05_31, get_core(store).query('store_id != 0')], ignore_index=True)

In [46]:
metrics_9_23_03_01__23_05_31.groupby(['line_item_id'], as_index=False).core.sum().query('core == 218')

,line_item_id,core
65,10029915,218
103,10051768,218
104,10051784,218
105,10051805,218
106,10052162,218
...,...,...
140100,923576,218
140102,923651,218
140103,923653,218
140105,923657,218


In [47]:
metrics_9_23_03_01__23_05_31.to_csv('metrics_9_23_03_01__23_05_31.csv')

In [48]:
del(metrics_9_23_03_01__23_05_31)